# 🩺 Machine Learning with Structured Patient Data

In this module, we will work through an illustrated example of using artificial intelligence to develop a patient risk prediction tool using electronic health records (EHR) data. *Adapted from Parisa Rashidi 2021, University of Florida Biomedical Data Science, Department of Biomedical Engineering.*

### 📚 After this module, students will be able to:

* Explain the lifecycle of typical machine learning (ML) projects.
* Preprocess datasets using Pandas implementations of common data pipelines.
* Train an XGBoost machine learning model using a structured patient dataset.
* Build a deep learning (DL) classifier using Keras.
* Explain model decision-making through interpretability techniques.

In this notebook, we will implement two machine learning models to predict the risk of acute kidney injury (AKI) following major inpatient surgery, using a tabular dataset of patient features.

# 🚀 Let's get started!

### 🤔 Motivation

* Predicting surgical complications *before* surgery can inform (1) shared decisions regarding the appropriateness of procedures, (2) targeted risk-reduction strategies, and (3) postoperative resource use. 

* Cognitive and judgment errors are major sources of potentially preventable complications.
    * Example: if the risk of complications is underestimated, high-risk patients may be triaged to general wards rather than more appropriate intensive care units (ICUs).
    
* AI-based clinical decision support has the potential to mitigate harm from cognitive errors occurring when estimating the risk of postoperative complications.

* All patients have a unique risk profile that is specific to their demographic characteristics, comorbid conditions, physiological reserve, planned surgical procedure, and surgeon skill.

* Clinicians sometimes display moderate ability to estimate risk probabilities. AI tools are intended to augment decision-making.

* Many clinicians are hindered by time-consuming manual data entry requirements and lack of integration with clinical workflow.
    
* AI predictive models using automated EHR data inputs may be able to mitigate these challenges.

### 🔄 Lifecycle of ML

Shown below are the major steps that are involved in a typical ML lifecycle.
<img src="https://github.com/gatorai/aicc23/raw/main/graphics/ML.png" alt="ML lifecycle" width="50%">

#### 1. Data extraction and processing
* Relevant data must be extracted (example: all revelant laboratory values and vital signs).
* Data quality and conformation to modeling requirements must be ensured through data preprocessing (example: fixing the missing data, removing any unwanted outliers, and encoding the categorical data.)

#### 2. Model development
* Machine learning models are built from data.
* Recall our module on functions: an ML model can be thought of as a **function** that maps a set of input values (example: patient characteristics) to an output value (example: probability of developing AKI).
* We develop a model on data from a **training** cohort.
* During the model refinement stage, known as **training**, model parameters (such as *weights* and *biases*) are altered to optimize the associations between inputs and outputs.
* After training, a model is ready to make predictions on *new data it has not seen before*. But how accurate will it be?

#### 3. Evaluation and Intepretation
* Once a ML model is trained, we evaluate the end result on **testing** dataset to see how well the model performed or how reliable it is.
* Multiple performance metrics such as accuracy, AUC, sensitivity, and specificity can be evaluated.
* Once we are satisfied with a model's performance, we may implement **explainable AI (XAI)** techniques to discover **why** a model behaves in a particular way.
* XAI techniques include computing a relative importance score associated with each input feature/column (for a given patient, how important was each input in determining that patient's prediction?).

#### 4. Deployment
* ML model deployment is the process of placing a finished ML model into a live environment, such as creating a web service or mobile application for prediction.
* It is imperative that the performance of an established model is continually monitored in prospective deployment to safeguard against data distribution shifts resulting from changes in populations, practice, or other changes that may result in deteriorating model performance over time.

#### 5. Optimization
* A deployed ML model has real-world impact and must be continuously monitored and maintained.
* When new or updated data becomes available, or when we determine that an existing deployed model no longer results in satisfactory performance, we need to optimize the model by re-training and re-evaluating the ML model.

In this notebook, we will only focus on the first three phases. 

## 📖 Data and problem definition

We will be using an EHR dataset containing 10,000 surgical patients.
* **📥 Input features (X)** (47)
    * Demographic variables
    * Admission and surgery information
    * Social determinants of health (SDOH)
    * One-year medication history
    * Admission comorbidities
    * Estimated glomerular filtration rate (eGFR: measures kidney function) before surgery
* **📤 Outcome (Y)** (1)
    * Postoperative AKI: onset of acute kidney injury (AKI) at any point between the end of surgery and patient discharge or death.
    * This is a binary outcome (it is recorded as either 0 or 1 for each patient). The ML model will learn to predict the risk of each patient developing AKI based on the specific input features.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('') # Files removed from public access
df.head()

💡 As an exploratory step, let's take a look at the distribution of our AKI outcome (how prevalent is postoperative AKI?). We'll use the `value_counts` function we saw before to help answer this question.

In [ ]:
# The normalize=True parameter divides each count by the total number of samples.
df['aki_surg_disch'].value_counts(normalize=True)

🔍 We can see from the above that about 16% of the surgical encounters resulted in acute kidney injury.

## ⚙️ Data preprocessing

Before we use our data to train a machine learning model, we must transform the raw data into **features** that an ML model can understand (sometimes these look quite similar, but not always). This critical step is called **data preprocessing**. Since real-world data can be messy, during this phase we also *clean* data to ensure the ML model can learn meaningful patterns.

Common data preprocessing steps may include the following (depending on the ML model used):
* Removing outliers*
* Addressing data missingess
* Transforming binary string columns (e.g., sex) into a numerical form that the model can understand.
* Transforming many-valued string columns (e.g., race) into a numerical form.
* Normalizing input features to give them equal weighting.

\*One common approach to remove outliers in ML projects is to remove observations that fall outside a specified range. In this tutorial, we remove observations whose values are less than the 1st percentile or greater than the 99th percentile.


### 🕵️‍♂️ Addressing missing data

* Missing data are common in routinely collected health data
    * Many times, missingness can be informative (e.g. a particular test was not ordered because patient was healthy).
    * In other cases, missingness may be at random or due to other factors (e.g., lack of routine follow-up)
* Some models **require** filling in missing data with imputation methods. Others will gladly accept missing data. The decision of when, and how, to impute missing data is often dependent on the model, data, and problem. 
* In a Pandas DataFrame, missing values will be represented by `NaN` (*"Not a Number"*)

One simple, but effective, imputation approach is to replace a patient's missing variable value with a value computed from all other patients. For example, if a patient is missing the `bmi` variable, we might impute the mean value for the `bmi` feature across all other patients. (Recall our Pandas module for other choices!)

**🔍 Example:** Let's use code to replace any missing `bmi` values with the `median` bmi from all other patients. We can accomplish this using some Pandas functions we've already seen, along with the `df.fillna(x)` function, which for a given DataFrame `df`, replaces any missing values with the input value `x`.

In [ ]:
# We can use Pandas filtering to see examples of where bmi is missing. (Notice the NaN)
df[pd.isnull(df['bmi'])].head()

In [ ]:
# Compute the median BMI (note: missing values are ignored in this calculation)
bmi_median = df['bmi'].median()
print(bmi_median)

In [ ]:
# Use fillna to replace any missing BMI values with the median BMI value.
df['bmi'] = df['bmi'].fillna(bmi_median)

In [ ]:
# Let's re-run this code to see where BMI is still missing
df[pd.isnull(df['bmi'])]

🎉 BMI is no longer missing! Every patient now has a BMI value for passing into the model.

**📌 Note:** There are many alternative approaches for missing data imputation, including k-Nearest neighbors (KNN) and multiple imputation by chained equation (MICE).

### ➡️ Data Transformation
In general, machine learning models are great at learning from **numerical** data (e.g., 0, 1, 0.5) . However, many important clinical characteristics are stored as **categorical** data (e.g., 'MALE', 'DIVORCED'). For many models, we must convert the categorical values to numerical features through a **data transformation** process.

For categorical variables with **only two** options, we can assign one of the options to be **0**, and the other to be **1**.

Let's see an example of how we would do this for the `sex` variable, which in our particular dataset contains only two values: `MALE` and `FEMALE`:

In [ ]:
# Show the unique values of the sex variable before transformation
print(df['sex'].unique())

# We can use the replace function to convert strings to binary numbers
df['sex'] = df['sex'].replace({'FEMALE' : 0, 'MALE' : 1})

# Show the unique values of the sex variable after transformation
print(df['sex'].unique())

🎉 Great! But what if our categorical variable contains more than two unique values?
* One option would be to replace them with more numbers (for example: 0, 1, 2, 3, ...)
* However, doing this may confuse some models due to implicit ordering and hierarchies
    * If 0=apple, 1=banana, 2=pear, 3=tomato, then does a tomato equal three bananas? Is a pear worth more than a banana which is worth more than an apple? 

📌 One solution is called **one-hot encoding**:
1. For a single categorical variable (e.g., `marital_status`), create a separate column in the dataset for each category of a categorical variable.
    * If there are 4 options, then we create 4 new columns.
2. The values of these columns are zero **except for** the column corresponding to the original value.

🔍 Let's see this in action:

In [ ]:
# Show the first 5 values of marital_status before one-hot encoding
print(df['marital_status'].head())

# Use the get_dummies() function to perform one-hot encoding on the marital_status column.
marital_status_transformed = pd.get_dummies(df['marital_status'])

# Show the first 5 values of marital_status after one-hot encoding
print(marital_status_transformed.head())

### ⏭️ Let's skip ahead.
Data analysis, imputation, transformation, and preprocessing take a large portion of any AI/ML project. We only covered a few steps of the process.

🕑 In in the interest of time, let's load a new dataset that has already been preprocessed.

In [ ]:
df = pd.read_csv('') # Files removed from public access
df.head()

## 🛠️ Model development

It's now time to train our first machine learning models!

We're going to look at two different models to predict postoperative AKI from our patient characteristics contained in `df`:
* **XGBoost:** A popular tree-based model that is known to work well with tabular datasets like ours.
* **Multi-layer perceptron (MLP):** A simple artificial neural network (ANN) model. This model is considered to be one of the simplest forms of **deep learning (DL)**.

### 🪚 Splitting the dataset

* **Recall** that ML models adjust their internal parameters by feeding in many examples of inputs $X$ and corresponding output $y$. This is called the **training** process. With enough time and training examples, the model can learn associations between inputs and outputs that can help it make a prediction on new data it hasn't ever seen before.

* To evaluate how good our ML model is, we will **test** it on a set of examples that it has never seen before (known as the **test set**), and for each patient in the test set, compare the model's predicted outcome $\hat{y}$ with the true patient outcome $y$.

* With enough testing examples, we can calculate performance metrics such as accuracy and other diagnostic metrics like AUC, sensitivity, and specificity.

🪛 Let's take our dataset and prepare it for building our ML model by doing a few final preprocessing items:
* Convert the pandas DataFrame into two NumPy arrays: `X` (inputs) and `y` (AKI outcome)
* Split X and y into training and testing sets. We'll have four variables after this step:
    * `X_train` and `y_train`: the training set
    * `X_test` and `y_test`: the testing set
    * We'll arbitrarily use 20% of the dataset for testing.

⚠️ Some of this code may seem unfamiliar to you. That's OK! Follow along with the comments to understand each step.

In [ ]:
from sklearn.model_selection import train_test_split # We'll use this function to create the training and testing sets

# Build X (an array/matrix) by taking the values from the entire df DataFrame EXCEPT FOR the first (patient id) and last (AKI outcome) columns
X = df.iloc[:, 1:-1].values

# Build y (a one-dimensional array) by taking only the final column ('aki_surg_disch'). The order of these outcomes exactly matches the order of the rows in X.
y = df.iloc[:, -1].values

# Split X and y into training and testing sets, by using a random 20% of the dataset for testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

## 🌲 Model 1: XGBoost

* Let's train our first XGBoost model to predict postoperative AKI from our patient features.
* We're going to use the popular Python library `xgboost`, which makes it very easy to train an ML model ***in a single line of code!***

In [ ]:
import xgboost as xgb

# Create our XGBoost prediction model and assign it to variable xgb
xgb = xgb.XGBClassifier()

# Train the model using the training dataset
xgb.fit(X_train, y_train)

🎉 That was easy! But is this model any good?

To evaluate the performance of our model, we need to compare two quantities:  
* $y$, the true outcomes of our testing set patients. In our code, we'll call this list `y_true`.
* $\hat{y}$, our model's **predicted** outcomes for the testing set patients. We'll call this `y_pred` (predicted).

📌 The XGBoost model has an easy function called `score` that computes the accuracy on a test dataset.

In [ ]:
# This will show our model's accuracy on the test set.
xgb.score(X_test, y_test)

### 🔍 Feature importance
* We now have our trained XGBoost model, and know how accurate it is at making predictions, but we don't understand **why** it predicts certain outcomes for certain patients.
* For this, we need to turn to the field of **explainable AI (XAI)**, an emerging and active field of AI research.
* In this module, we'll examine one of the simplest forms of explainability, which looks at the model's weight it assigns to each feature.
* Every XGBoost model has a built-in *attribute* variable for this: `feature_importances_` (for convenience, we'll reassign the feature importances to a variable `F`).

In [ ]:
# Compute feature importances
F = xgb.feature_importances_

# Take a look at the feature importances.
# The scores are in the order of the columns of our input data.
print(F)

* To make this more informative, we can map these scores to their original feature names and generate a plot to visually compare the importance of each feature.
* Let's do this for the model's top 10 features.

**⚠️ Note:** Some of this code will be new. It's OK to just follow along!

In [ ]:
F_df = pd.DataFrame({'feature': df.columns.tolist()[1:-1], 'score': F}).sort_values(by='score').tail(10)
F_df.plot.barh(x='feature', y='score', figsize=(5,3))

* By examining the model's learned weights, we can see that the most influential input was the `surgery_type` feature (further analysis can reveal which surgery types were most predictive of AKI)
* The second most important feature was `lytes`, a comorbidity indicating fluid and electrolyte disorders.

## 🤖 Model 2: Artificial Neural Network (Multi-Layer Perceptron)
* An important part of an AI research project is evaluating different models to identify which works best for a given dataset and task.
* Let's train a different machine learning model on the same dataset.
* This time, we'll be using a deep learning (DL) model that uses an artificial neural network called the multi-layer perceptron (MLP).
* We will be using the user-friendly Python library `Keras` to build our MLP model.

**⚠️ Note:** A full description of DL or the MLP is beyond the scope of this lesson. We refer interested students to [Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep learning. MIT press](https://www.deeplearningbook.org).

📌 Important bullet points for this module:
* Deep learning was biologically inspired.
* A DL model like the MLP is often defined in terms of **layers** (one *input layer*, one or more *hidden layers*, and one *output layer*). The more hidden layers that are added to a model, the "deeper" it gets.
* Each hidden layer is designed to transform the data from the layer before.
* The final layer learns to predict an outcome based on all of these transformations.

<img src="https://github.com/gatorai/aicc23/raw/main/graphics/ANN.jpg" alt="ANN" width="600" height="600">

🛠️ Let's create our first MLP model with the following specification:
* One input layer with 47 units (one per input variable)
* One hidden layer with 128 units
* Another hidden layer with 64 units
* An output layer with 1 unit (for predicting 0 or 1 corresponding to our AKI outcome)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Sequential

model = Sequential()
# Add the first hidden layer with 128 neurons (and implicitly create the input layer by specifying the "input_dim")
model.add(layers.Dense(units=128, input_dim=47, activation='relu'))

# Add the second hidden layer with 64 neurons
model.add(layers.Dense(units=64, activation='relu'))

# Add the output layer
model.add(layers.Dense(units=1, activation='sigmoid'))

# Compile the model for a classification problem such as ours.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 💬 (Optional) Code Explanation

Each hidden layer of our neural network will be created using the **Dense** class from Keras. For each layer, we must define the number of hidden units (also known as neurons). There are several optional arguments we may also pass, which can be viewed in the [Keras documentation page](https://keras.io/api/layers/core_layers/dense/). We can add many layers to our deep learning model using the .add() function of the Sequential class. You can think of a Sequential container as a list of hidden layers.

For the first layer of our neural network, we must tell Keras how many variables to expect in each input vector. From our previous data exploration, we know that each patient is defined by 47 different variables, so the input dimension to our network is 47.

One reason why deep learning models are so powerful is their ability to model complex variable interactions through nonlinear activation functions. We have several choices for activation function. In our example, we will use the commonly chosen Rectified Linear Unit activation (ReLU).

Once we are satisfied with the hidden layers of our model, we need to add an output layer for generating class predictions. Our output layer will also be a Dense layer, but it will only have a single (1) unit. Instead of ReLU, we will use a sigmoid activation function, which is typically chosen for binary classification problems such as ours. Using a sigmoid activation on our output layer allows us to interpret the output as a prediction probability. In other words, the probability that a given input vector belongs to class 1.

Now that we have defined the architecture of our neural network, we will use the .compile() function to build it. In our example we are defining a few arguments that are associated with the training of our model:
* We are using a binary cross-entropy loss. This is an appropriate choise for binary classification.
* We will be using the Adam optimizer, which is a popular version of stochastic gradient descent (SGD).
* For this example, we are interested in our model's prediction accuracy, so we'll tell Keras to use the "accuracy" metric. 

### 🏋️‍♂️ Training our MLP model

* Now it's time to train our prediction model!
* We will train (also called "fit") the model using our training dataset we already created.
* We will use the one-line function `.fit()` to train our entire deep learning model.
* We will specify some additional parameters to be used during the training process:
    * We will tell Keras to train the model for 10 epochs.
    * We will use a batch size of 64 samples. During each epoch, the model will pass in 64 samples at a time.
    * We will use a random 30% of the training dataset as our **validation set** (different from the test set) for computing metrics while training.

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split= 0.3)

**🎉 Done!**

* Let's check the performance of our trained model on the test set we already created.
* The model has never seen this particular data, so it can provide an idea of how well the model might perform in the future (***generalizability to unseen data***).
* We will use the Keras function `.evaluate()`, which will compute the loss, as well as any metrics we defined when compiling our model.
    * Since we told Keras to use "accuracy" when we compiled the model, we will see the model's accuracy on the test data.

In [ ]:
scores = model.evaluate(X_test, y_test)

**✏️ Excercise:** Create an MLP with **3** hidden layers (instead of 2), with the following number of neurons in each:
* Hidden layer 1: 512 neurons
* Hidden layer 2: 256 neurons
* Hidden layer 3: 128 neurons

Create, compile, train, and evaluate this new MLP model.

In [ ]:
# Code it! (feel free to re-use most of the code from before!)


**✏️ Excercise:** Create an MLP with **8** hidden layers (instead of 2), with **any number** of neurons in each hidden layer. Create, compile, train, and evaluate this new MLP model.

In [ ]:
# Code it!
